In [ ]:
import random 
import gym 
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.optimizers import Adam
import os 

In [ ]:
# creating the env
env = gym.make('CartPole-v0')

/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:594: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  f"The environment {id} is out of date. You should consider "
/usr/local/lib/python3.7/dist-packages/gym/core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in futu

In [ ]:
n_episodes = 1000
batch_size = 32
state_size = 4

In [ ]:
output_dir = '/content/cartpole'
if not os.path.exists(output_dir) :
  os.makedirs(output_dir)

In [ ]:
from keras.optimizers import Optimizer
from keras.models import sequential
class DQN : 

  def __init__(self, state_size, action_size) :
    self.state_size = state_size
    self.action_size = action_size 
    self.memory = deque(maxlen=2000)
    self.gamma = 0.95 
    self.epsilon = 1.0 
    self.epsilon_decay = 0.995
    self.epsilon_min = 0.01
    self.learning_rate = 0.001 
    self.model = self._build_model()

  
  def _build_model(self) :

    model = Sequential()
    model.add(Dense(24, input_dim = self.state_size, activation = 'relu'))
    model.add(Dense(24, activation = 'relu'))
    model.add(Dense(self.action_size, activation = 'linear'))
    model.compile(loss='mse', optimizer = Adam(lr = self.learning_rate))

    return model 
  
  def remember(self, state, action, reward, next_state, done) :
    self.memory.append((state, action, reward, next_state, done))
  
  def act(self, state) :
    if random.random() < self.epsilon :
      return random.randrange(self.action_size)
    act_values = self.model.predict(state)
    return np.argmax(act_values[0])
  
  def replay(self, batch_size) :
    minibatch = random.sample(self.memory, batch_size)
    for state, action, reward, next_state, done in minibatch :
      target = reward 
      if not done :
        target = reward + self.gamma*(np.argmax(self.model.predict(next_state)[0]))
      target_f = self.model.predict(state)
      target_f[0][action] = target 

      self.model.fit(state, target_f, epochs = 1, verbose = 0)

    self.epsilon = max(self.epsilon*self.epsilion_deccay, self.epsilon_min)

  def load(self, name) :
    self.model.load_weights(name)

  def save(self, name) :
    self.model.save_weights(name) 

In [ ]:
# for cartpole 
agent = DQN(4, 2)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
done = False 

for e in range (n_episodes) :
  state = env.reset()
  state - np.reshape(state, [1, state_size])

  for time in range (5000) :
    # env.render()
    action = agent.act(state)
    next_state, reward, done, _ = env.step(action)

    reward = reward if not done else -10 

    next_state  = np.reshape(next_state, [1, state_size])

    agent.remember(state, action, reward, next_state, done)

    state = next_state

    if done :
      print("episode: {}/{} score: {}, e: {:.2}".format(e, n_episodes, time, agent.epsilon))
      break 
    

  if len(agent.memory) > batch_size :
    agent.replay(batch_size)
  
  if e%50 == 0 :
    agent.save(output_dir + "weights_" + '{:04d}'.format(e) + ".hdf5")


episode: 0/1000 score: 27, e: 1.0


ValueError: ignored